# Otica Watchlist

Importion Libraries

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from openpyxl import Workbook
import re
import socket
import psutil
import plotly.express as px
import plotly.graph_objects as go 
from dash import Dash, html, dcc, dash_table, Input, Output
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import dash.dependencies as dd
from dash.dash_table.Format import Group
from plotly.subplots import make_subplots

Yahoo finance alternative API calls. https://github.com/ranaroussi/yfinance/issues/2422#issuecomment-2840774505 \n https://github.com/ranaroussi/yfinance/pull/2277

In [5]:
# self._set_session(session or requests.Session(impersonate="chrome"))

#  from curl_cffi import requests
#  session = requests.Session(impersonate="chrome")
#  ticker = yf.Ticker('...', session=session)

## Enter Data Here

In [91]:
# --- User-defined dynamic parameters ---
symbols = ['O', 'AMZN']               # Ticker symbols to fetch
dates = [30, 100, 200]                # Periods for MA and Bollinger Bands
period_range = "10y"                 # Data time range: last 10 years
auto_adjust_prices = True            # Whether to auto-adjust for splits/dividends
rsi_period = 14                      # RSI calculation window
# --------------------------------------

accent

In [52]:
accent_color = '#1abc9c'

In [21]:
# Initialize an empty dictionary to store results
symbol_dataframes = {}

for symbol in symbols:
    try:
        print(f"Fetching data for {symbol}...")
        data = yf.download(symbol, period=period_range, auto_adjust=auto_adjust_prices)

        # --- Dynamic Moving Averages & Bollinger Bands ---
        for period in dates:
            ma_col = f'MA{period}'
            std_col = f'BB{period}_Std'
            upper_col = f'BB{period}_Upper'
            lower_col = f'BB{period}_Lower'

            data[ma_col] = data['Close'].rolling(window=period).mean()
            data[std_col] = data['Close'].rolling(window=period).std()
            data[upper_col] = data[ma_col] + (2 * data[std_col])
            data[lower_col] = data[ma_col] - (2 * data[std_col])

        # Drop standard deviation columns
        std_cols = [f'BB{period}_Std' for period in dates]
        data.drop(columns=std_cols, inplace=True)

        # --- MACD ---
        data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
        data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
        data['MACD'] = data['EMA12'] - data['EMA26']
        data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()
        data['MACD_Histogram'] = data['MACD'] - data['Signal_Line']

        # --- RSI ---
        delta = data['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=rsi_period, min_periods=rsi_period).mean()
        avg_loss = loss.rolling(window=rsi_period, min_periods=rsi_period).mean()
        rs = avg_gain / avg_loss
        data['RSI'] = 100 - (100 / (1 + rs))

        symbol_dataframes[symbol] = data
    except Exception as e:
        print(f"Failed to fetch data for {symbol}: {e}")

# Cleanup MultiIndex columns if needed
for symbol, df in symbol_dataframes.items():
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

Fetching data for O...


[*********************100%***********************]  1 of 1 completed


Fetching data for AMZN...


[*********************100%***********************]  1 of 1 completed


## Wathclist

In [118]:
def generate_stock_chart(symbol):

    # pull your df
    df = symbol_dataframes[symbol].reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    # signed-log MACD
    df['MACD_slog']      = np.sign(df['MACD']) * np.log1p(np.abs(df['MACD']))
    df['Signal_slog']    = np.sign(df['Signal_Line']) * np.log1p(np.abs(df['Signal_Line']))
    df['MACD_Hist_slog'] = np.sign(df['MACD_Histogram']) * np.log1p(np.abs(df['MACD_Histogram']))

    # split volume
    threshold = df['Volume'].quantile(0.95)
    max_v     = df['Volume'].max()
    df['Vol_bot'] = np.minimum(df['Volume'], threshold)
    df['Vol_top'] = np.where(df['Volume'] > threshold, df['Volume'] - threshold, 0)

    # make the subplots
    fig = make_subplots(
        rows=6, cols=1,
        shared_xaxes=True,
        row_heights=[0.55, 0.02, 0.15, 0.126, 0.014, 0.14],
        vertical_spacing=0.03,
        specs=[
            [{}], [None], [{}],
            [{}], [{}],
            [{}]
        ]
    )

    # Row 1: Close + MAs + BBands
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['Close'], mode='lines',
        name='Close',
        line=dict(color=accent_color, width=2)
    ), row=1, col=1)

    for idx, p in enumerate(dates):
        ma = f'MA{p}'
        if ma not in df:
            continue
        color = 'lightgrey' if idx == len(dates)//2 else accent_color
        fig.add_trace(go.Scatter(
            x=df['Date'], y=df[ma], mode='lines',
            name=ma,
            line=dict(color=color, width=1.5)
        ), row=1, col=1)

    greys = [f'#{v:02x}{v:02x}{v:02x}' for v in np.linspace(128,200,len(dates),dtype=int)]
    for p, grey in zip(dates, greys):
        up, lo = f'BB{p}_Upper', f'BB{p}_Lower'
        if up in df and lo in df:
            fig.add_trace(go.Scatter(
                x=df['Date'], y=df[up], mode='lines',
                name=up,
                line=dict(color=grey, dash='dot')
            ), row=1, col=1)
            fig.add_trace(go.Scatter(
                x=df['Date'], y=df[lo], mode='lines',
                name=lo,
                line=dict(color=grey, dash='dot')
            ), row=1, col=1)

    fig.update_yaxes(title_text="Price", row=1, col=1)

    # Row 3: RSI
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['RSI'], mode='lines',
        name='RSI',
        line=dict(color=accent_color, width=1.5)
    ), row=3, col=1)
    fig.update_yaxes(range=[0,100], title_text="RSI", row=3, col=1)
    fig.add_hline(y=70, line=dict(color='grey', dash='dash'), row=3, col=1)
    fig.add_hline(y=30, line=dict(color='grey', dash='dash'), row=3, col=1)
    fig.add_annotation(xref='paper', x=1, xanchor='right',
                       yref='y3', y=70, text='Overbought',
                       font=dict(color='grey'), showarrow=False, yshift=8)
    fig.add_annotation(xref='paper', x=1, xanchor='right',
                       yref='y3', y=30, text='Oversold',
                       font=dict(color='grey'), showarrow=False, yshift=8)

    # Row 4: Volume bottom (accent color)
    fig.add_trace(go.Bar(
        x=df['Date'], y=df['Vol_bot'],
        name='Volume',
        marker_color = accent_color
    ), row=4, col=1)
    fig.update_yaxes(
        range=[0, threshold],
        title_text="Volume",
        tickmode='array',
        tickvals=[threshold],
        ticktext=[f"{int(threshold):,}"],
        showgrid=False,
        row=4, col=1
    )


    # Row 6: MACD + hist
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['MACD_slog'], mode='lines',
        name='MACD',
        marker_color = 'blue',
        line=dict(width=1.5)
    ), row=6, col=1)
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['Signal_slog'], mode='lines',
        name='Signal',
        marker_color = 'red',
        line=dict(width=1.5)
    ), row=6, col=1)
    fig.add_trace(go.Bar(
        x=df['Date'], y=df['MACD_Hist_slog'],
        name='Hist',
        marker_color=['green' if v >= 0 else 'red' for v in df['MACD_Hist_slog']]
    ), row=6, col=1)
    fig.update_yaxes(title_text="MACD", row=6, col=1)
    fig.add_hline(y=0, line=dict(color='grey', dash='dash'), row=6, col=1)

    # X-axes and layout tweaks
    fig.update_xaxes(row=1, col=1, showticklabels=True)
    for r in (3,4,5,6):
        fig.update_xaxes(row=r, col=1, showticklabels=False)

    fig.update_layout(
        template='plotly_dark',
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#1e1e1e',
        font_color=accent_color,
        margin=dict(t=20, b=20, l=20, r=20),
        height=950,
        legend=dict(traceorder='grouped',
                    tracegroupgap=10,
                    orientation='v',
                    x=1.02, y=1)
    )

    return fig

## Dashboard

In [121]:
import socket
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# -----------------------------------------------------------------------------
# Helper to get a free localhost port
# -----------------------------------------------------------------------------
def get_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('127.0.0.1', 0))
    _, port = s.getsockname()
    s.close()
    return port

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# Dash app: Watchlist Visualization + callback to change stocks
# -----------------------------------------------------------------------------
app = Dash(__name__)
port = get_free_port()

app.layout = html.Div(
    style={'backgroundColor':'black','color':'white','padding':'20px'},
    children=[
        html.H1("Watchlist Visualization", style={'color':accent_color}),
        dcc.Dropdown(
            id='watchlist-dropdown',
            options=[{'label': s,'value':s} for s in symbol_dataframes.keys()],
            value=list(symbol_dataframes.keys())[0],
            clearable=False,
            style={'width':'300px'}
        ),
        dcc.Graph(id='watchlist-chart', style={'height':'900px'})
    ]
)

@app.callback(
    Output('watchlist-chart','figure'),
    [Input('watchlist-dropdown','value')]
)
def update_watchlist_chart(symbol):
    return generate_stock_chart(symbol)

# -----------------------------------------------------------------------------
# Run inline in notebook
# -----------------------------------------------------------------------------
app.run_server(mode='inline', debug=True, port=port)
print(f"App running at http://127.0.0.1:{port}")


App running at http://127.0.0.1:53179
